In [ ]:
pip install -r requirements.txt

# Prerequisites

Follow the steps in descried in [bpmn_bbo_mapping](https://github.com/MartijnBogaert/bpmn_bbo_mapping) to create an `input.nq` file from a bpmn.

# Run mapping
Note: change `--pass` to `--pass-only-new` to only keep newly derived triples. i.e. drop the input triples. In a later implementation `--query` should be used to only keep the relevant triples. For now keeping everything is easier during development.

In [3]:
! project_path=$(cd ../ && pwd) ; \
  docker run \
    --net=host --rm \
    -v $project_path:$project_path \
    -w $(pwd) \
    eyereasoner/eye \
      --nope --quiet --pass \
      input.nq \
      https://www.irit.fr/recherches/MELODI/ontologies/BBO/ontology.ttl \
      ../bbo-to-oslo-steps.n3 \
        > oslo-steps.ttl

## Optional - pretty print result
making the output more readible.

In [ ]:
! ./pretty_turtle oslo-steps.ttl

# Exploration

Used during development - execute sparql queries on graph.

In [ ]:
import rdflib
import pandas as pd
from io import StringIO


In [ ]:
g = rdflib.Graph()

g.parse('oslo-steps.ttl', format='ttl')

In [ ]:
def sparql_result_to_dataframe(sparql_result):
    return pd.read_csv(StringIO(sparql_result.serialize(format='csv').decode('utf-8')))

In [ ]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX tai: <https://www.teamingai-project.eu/> 
PREFIX bbo: <https://www.irit.fr/recherches/MELODI/ontologies/BBO#> 
PREFIX bbo-ex: <https://www.teamingai-project.eg/BBOExtension#> 
PREFIX o-steps: <https://fast.ilabt.imec.be/ns/oslo-steps#> 
PREFIX string: <http://www.w3.org/2000/10/swap/string#> 
PREFIX xl: <http://www.w3.org/2008/05/skos-xl#> 
SELECT distinct ?p {
  ?activity
    rdfs:subClassOf
      bbo:Activity.

  ?incoming
    bbo:has_targetRef
      ?activity;
    bbo:has_sourceRef
      ?requiredState.

  ?requiredState
    rdfs:subClassOf
      bbo:Activity;
    ?p ?o.                 
}
""")

# Output result
sparql_result_to_dataframe(result)

# Findings:
Not all TextAnnotations are linked to something. There seem to be floating "Ja" and "Nee" annotations

In [ ]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX tai: <https://www.teamingai-project.eu/> 
PREFIX bbo: <https://www.irit.fr/recherches/MELODI/ontologies/BBO#> 
PREFIX bbo-ex: <https://www.teamingai-project.eg/BBOExtension#> 
PREFIX o-steps: <https://fast.ilabt.imec.be/ns/oslo-steps#> 
PREFIX string: <http://www.w3.org/2000/10/swap/string#> 
PREFIX xl: <http://www.w3.org/2008/05/skos-xl#> 
SELECT distinct * {
   ?annotation
      rdfs:subClassOf bbo-ex:TextAnnotation;
      rdfs:comment ?comment. 

   FILTER NOT EXISTS {
      ?association
         rdfs:subClassOf
            bbo-ex:Association;
         bbo:has_targetRef
            ?annotation.
   }              
}
""")

sparql_result_to_dataframe(result)

# Cleanup

In [ ]:
! rm *.ttl *.pkl